<pre>
Copyright 2021-2022 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to enable
# nested event loops. We don't need that when using `isabelle-client` from
# Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
from isabelle_client import start_isabelle_server
# first, we start Isabelle server
server_info, _ = start_isabelle_server(
    name="test", port=9999, log_file="server.log"
)

In [3]:
import logging

from isabelle_client import get_isabelle_client
# then we create Python client to Isabelle server
isabelle = get_isabelle_client(server_info)
!rm -f session.log
# we will log all the messages from the server to a file
isabelle.logger = logging.getLogger()
isabelle.logger.setLevel(logging.INFO)
isabelle.logger.addHandler(logging.FileHandler("session.log"))

In [4]:
# suppose we have an Isabelle theory file
# (probably generated by another Python script)
!cat Example.thy

theory Example
imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by auto
end

In [5]:
# we can send this file to the server and get a response
isabelle.use_theories(
    theories=["Example"], master_dir=".", watchdog_timeout=0
)

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":14,"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5","message":"theory Draft.Example 14%","kind":"writeln","session":"","theory":"Draft.Example"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5","message":"theory Draft.Example 99%","kind":"writeln","session":"","theory":"Draft.Example"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5","message":"theory Draft.Example 100%","kind":"writeln","session":"","theory":"Draft.Example"}', response_length=163),
 IsabelleResponse(response_

In [6]:
# if we have a session description using ROOT file like this
!rm -rf output
!cat ROOT

session examples = HOL +
  options [document = pdf, document_output = "output"]
  theories
    Example
  document_files
    "root.tex"


In [7]:
# we can build session (`isabelle build`)
isabelle.session_build(dirs=["."], session="examples")

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"cf68d431-68b5-4192-a0f8-a953efd345e6"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Building examples ...","task":"cf68d431-68b5-4192-a0f8-a953efd345e6"}', response_length=104),
 IsabelleResponse(response_type='NOTE', response_body='{"task":"cf68d431-68b5-4192-a0f8-a953efd345e6","message":"examples: theory examples.Example","kind":"writeln","session":"examples","theory":"examples.Example"}', response_length=165),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Preparing examples/document ...","task":"cf68d431-68b5-4192-a0f8-a953efd345e6"}', response_length=114),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Finished examples/document (0:00:05 elapsed t

In [8]:
# the results will appear in `output` folder
# notice that `output` folder does not contain a PDF file
# because `build` command failed
# (this container has no LaTeX installed)
!ls output

document  document.pdf


In [9]:
import asyncio

# or we can issue a free-text command through TCP
# here `asynchronous` argument means 'asynchronous command' as defined
# in section 4.2.6. of Isabelle System manual. `echo` is a synchronous command
# but Python communicates with Isabelle asynchronously as always
asyncio.run(isabelle.execute_command("echo 42", asynchronous=False))

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='42', response_length=None)]

In [10]:
# we can also shut down the Isabelle server
isabelle.shutdown()

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='', response_length=None)]

In [11]:
# all our communications with the server
# were written to the log file specified earlier
!cat session.log

7d362cae-6983-4288-89b4-2ff07a56f0e9
session_start {"session": "HOL"}

OK {"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}
OK {"task":"5da4edfe-689a-4571-895e-f28ec7fa3e42"}
107
NOTE {"kind":"writeln","message":"Starting session HOL ...","task":"5da4edfe-689a-4571-895e-f28ec7fa3e42"}
175
FINISHED {"session_id":"9b2cf476-7e3b-4155-9e71-1724610cb561","tmp_dir":"/tmp/isabelle-boris/server_session4796758055019481326","task":"5da4edfe-689a-4571-895e-f28ec7fa3e42"}
7d362cae-6983-4288-89b4-2ff07a56f0e9
use_theories {"session_id": "9b2cf476-7e3b-4155-9e71-1724610cb561", "theories": ["Example"], "watchdog_timeout": 0, "master_dir": "."}

OK {"isabelle_id":"1ac2416e8432","isabelle_name":"Isabelle2022"}
OK {"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5"}
161
NOTE {"percentage":14,"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a5","message":"theory Draft.Example 14%","kind":"writeln","session":"","theory":"Draft.Example"}
161
NOTE {"percentage":99,"task":"0151b5e4-7442-4574-a567-a5dfc8d1a7a